In [1]:
import os
import sys
import json
import stanza
from tqdm import tqdm_notebook as tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work/ser'

In [2]:
sys.path.append(w_dir+'/ser_annotate')

In [3]:
from ser_annotate.std import *
from ser_annotate import config
from ser_annotate.dataset_reader.io_cano import CanoDict, CrpCano

2020-05-29 18:35:38 INFO: "zh" is an alias for "zh-hans"
2020-05-29 18:35:38 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package   |
-------------------------
| tokenize  | gsdsimp   |
| ner       | ontonotes |

2020-05-29 18:35:38 INFO: Use device: cpu
2020-05-29 18:35:38 INFO: Loading: tokenize
2020-05-29 18:35:38 INFO: Loading: ner
2020-05-29 18:35:38 INFO: Done loading processors!


In [58]:
test_file = config.SSQA_ANNOT / "PubB-G6a-0302.json1"

In [13]:
from ser_annotate.dataset_reader.io_SSQA import str_lesson2parags, str_lesson2st

In [66]:
def annot2se(annot_fp):
    def has_keyword(s, labels):
        for label in labels:
            assert label[2] == 'Keyword'
            l_start = label[0]
            l_end = label[1] 
            if s[0] <= l_start <= s[1]:
                assert s[0] <= l_end <= s[1], 'keyword cross sentence!!'
                return True
        return False             
    
    annots = jseq_load(test_file)
    myCrpCano = CrpCano()
    myCrpCano.load_annot_ssqa(annots)
    out_q_list = []
    for qid, q_annot in myCrpCano.items():
        se = []
        st_parags = str_lesson2st(q_annot.c_text)
        for p_i, p in enumerate(st_parags):
            for s_i, s in enumerate(p[2]):
                if has_keyword(s, q_annot.labels):
                    se.append([p_i, s_i])
#         assert se, "No supporting evidence in qid:{}".format(qid)
        out_q_list.append({'qid': qid, 'qtext': q_annot.qtext, 'context': st_parags, 'se': se})
    return out_q_list

In [67]:
out = annot2ser(test_file)

In [68]:
out[0]

{'qid': 'PubB-G6a-0302.jsonl-06-002408',
 'qtext': '「文文民調中心」問卷調查目前臺灣的家庭型態，發現「大家庭」是目前臺灣社會主要的家庭結構型態。',
 'context': [[0, 9, [[0, 9, '第三單元社會的變遷']]],
  [12, 19, [[12, 19, '家庭兩性新關係']]],
  [22, 31, [[22, 31, '＜家庭型態的轉變＞']]],
  [34, 64, [[34, 50, '工業化不但改變了我們的生活方式，'], [52, 64, '連帶的也影響家庭的型態。']]],
  [67,
   140,
   [[67, 78, '首先是居住型態的改變。'],
    [80, 92, '以前農業社會中的大家庭，'],
    [94, 102, '現在已經很少見，'],
    [104, 120, '取而代之的是三代同堂的折衷家庭，'],
    [122, 140, '和一對夫婦與未婚子女所組成的小家庭。']]],
  [143,
   315,
   [[143, 147, '近年來，'],
    [149, 162, '由於離婚或意外事故等因素，'],
    [164, 174, '產生了許多單親家庭；'],
    [176, 185, '而父母到都市就業，'],
    [187, 211, '將小孩託付給祖父母照顧的隔代教養家庭也時有所見；'],
    [213, 233, '而因異國婚姻所組成的家庭也有增加的趨勢。'],
    [235, 249, '這些家庭在經濟和子女教養上，'],
    [251, 261, '需要社會更多的關注，'],
    [263, 266, '因此，'],
    [268, 285, '政府應積極訂定完善的社會福利制度，'],
    [287, 297, '設立足夠的托兒機構，'],
    [299, 315, '協助家庭解決年幼子女的教養問題。']]],
  [318, 333, [[318, 333, '夫婦與未婚子女所組成的小家庭。']]],
  [336, 361, [[336, 341, '由祖父母、'], [343, 361, '父母與子女所組成三代同堂的折衷家庭。']]],
  [364, 38

# Create check excel

In [29]:
from pathlib import PurePath
from ser_annotate.dataset_reader.io_SSQA import xml2st

In [27]:
PurePath('/media/moju/data/work/ser/ser_annotate/data/SSQA/Elementary_Social_Studies_v3.0/Test/PubB-G3a-0301.xml').stem

'PubB-G3a-0301'

In [30]:
myFid, out_parags, myLessonTxt, q_dict = xml2st('/media/moju/data/work/ser/ser_annotate/data/SSQA/Elementary_Social_Studies_v3.0/Test/PubB-G3a-0301.xml')

In [31]:
myFid

'PubB-G3a-0301'

In [34]:
q_dict.keys()

dict_keys(['true-false', 'multiple-choice', 'multiple-select'])

In [48]:
test_fps = list_fps(config.SSQA_XML_DEV)

In [49]:
df_list = []
for fp in test_fps:
    myFid, out_parags, myLessonTxt, q_dict = xml2st(fp)
    df_list.append([myFid, 'None', 'False', 0, len(q_dict['true-false']), len(q_dict['multiple-choice']), len(q_dict['multiple-select'])])

In [50]:
import pandas as pd

In [51]:
df_check = pd.DataFrame(df_list ,columns=['did', 'Annotator', 'check', 'Time', 'true-false', 'multiple-choice', 'multiple-select'])

In [52]:
df_check.to_excel("check_dev.xlsx")  